In [7]:
from pathlib import Path
from argparse import ArgumentParser
import pandas as pd

experiment_dir = 'output'
exp_dir = Path(experiment_dir)
distinguishers = [fp for fp in exp_dir.iterdir() if str(fp).endswith('csv')]
graph_data = []
for distinguisher_fp in distinguishers:
    distinguisher_data = pd.read_csv(distinguisher_fp)
    unlearning_approaches = distinguisher_data['model'].unique()
    assert 'control' in unlearning_approaches
    distinguisher_name = '_'.join(distinguisher_fp.name.split('_')[:-1])
    control_data = distinguisher_data[distinguisher_data['model'] == 'control']
    for ul_approach in unlearning_approaches:
        if ul_approach == 'control':
            continue
        else:
            ul_data = distinguisher_data[distinguisher_data['model'] == ul_approach]
            greater_than = (ul_data['score'].values > control_data['score'].values).sum() / len(ul_data['score'].values)
            less_than = (ul_data['score'].values < control_data['score'].values).sum() / len(ul_data['score'].values)
            advantage = (max(greater_than, less_than)- 0.5) * 2
            row_data = {
                'distinguisher': distinguisher_name,
                'advantage': advantage,
                'unlearning': ul_approach
            }
            graph_data.append(row_data)
graph_df = pd.DataFrame.from_dict(graph_data)

In [8]:
graph_df

,distinguisher,advantage,unlearning
0,consistency,0.1,bad_teacher
1,consistency,0.9,amnesiac
2,kld,1.0,bad_teacher
3,kld,1.0,amnesiac
4,mia,1.0,bad_teacher
5,mia,1.0,amnesiac
6,mse,1.0,bad_teacher
7,mse,1.0,amnesiac
8,randomness,1.0,bad_teacher
9,randomness,1.0,amnesiac


In [9]:
import plotly.express as px
fig = px.bar(graph_df, x="distinguisher", y="advantage", 
                color="unlearning", barmode="group")
fig.show()